# Credit Risk Analysis Model Training Notebook

This notebook contains our process to training our model, our attempted optimizations, and our final results. 


#### Note

*This notebook requires access to 2 csv files. Please make sure you upload these to your google colab environment before continuing.*

*If you are using your own resources, please ensure you update the paths to your local directories.*

In [ ]:
!pip install keras-tuner

In [2]:
#import dependencies
import pandas as pd


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

pd.set_option('display.max_columns', None)

In [12]:
#load in the data
application_df = pd.read_csv('application_record.csv')
credit_record_df = pd.read_csv("credit_record.csv")

In [13]:
application_df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [14]:
credit_record_df.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [15]:
credit_df = pd.merge(application_df, credit_record_df, on='ID', how='inner')

In [ ]:
# 0: 1-29 days past due
# 1: 30-59 days past due
# 2: 60-89 days overdue
# 3: 90-119 days overdue
# 4: 120-149 days overdue
# 5: Overdue or bad debts, write-offs for more than 150 days
# C: paid off that month
# X: No loan for the month

In [16]:
#map our target column based on the STATUS column.
credit_df['target'] = credit_df['STATUS'].apply(lambda x: 0 if x in ['0','1','2', '3', '4', '5'] else 1)

In [17]:
#Remove Identifying column and column we used to determine the target.
credit_df.drop(columns=['ID',"STATUS"], inplace=True)
credit_df.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,target
0,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0,1
1,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-1,1
2,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-2,1
3,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-3,1
4,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-4,1


begin model training

In [18]:
df = credit_df.copy()

df = pd.get_dummies(df, columns=['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE'], drop_first=True)

df.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,MONTHS_BALANCE,target,CODE_GENDER_M,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff
0,0,427500.0,-12005,-4542,1,1,0,0,2.0,0,1,True,True,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,0,427500.0,-12005,-4542,1,1,0,0,2.0,-1,1,True,True,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,0,427500.0,-12005,-4542,1,1,0,0,2.0,-2,1,True,True,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,0,427500.0,-12005,-4542,1,1,0,0,2.0,-3,1,True,True,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,0,427500.0,-12005,-4542,1,1,0,0,2.0,-4,1,True,True,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [19]:
y = df['target']

X = df.drop(columns=['target'])

In [20]:
y.head()

,target
0,1
1,1
2,1
3,1
4,1


In [ ]:
X.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,MONTHS_BALANCE,CODE_GENDER_M,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff
0,0,427500.0,-12005,-4542,1,1,0,0,2.0,0,True,True,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,0,427500.0,-12005,-4542,1,1,0,0,2.0,-1,True,True,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,0,427500.0,-12005,-4542,1,1,0,0,2.0,-2,True,True,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,0,427500.0,-12005,-4542,1,1,0,0,2.0,-3,True,True,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,0,427500.0,-12005,-4542,1,1,0,0,2.0,-4,True,True,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


# Neural Network


Pre-processing

In [24]:
#split our dataset into the features and the target
y = df.target.values
X = df.drop(columns=["target"])

Train Test Split

In [25]:
#Split our data for training
x_train, x_test, y_train, y_test = train_test_split(
    X,
    y,
    random_state=1,
    stratify=y
)

Scaling Our Data

In [26]:
#Scale our data
scaler = StandardScaler()

x_scaler = scaler.fit(x_train)


x_train_scaled = x_scaler.transform(x_train)
x_test_sclaed = x_scaler.transform(x_test)

## First Model Configuration

In [66]:
#Begin Model Creation
model_1 = tf.keras.models.Sequential()
model_1.add(tf.keras.layers.Dense(units=1, activation="relu", input_dim=len(df.columns)-1))
model_1.add(tf.keras.layers.Dense(units=10, activation="relu"))
model_1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Compile the Sequential model together and customize metrics
model_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [67]:
model_1_fitted = model_1.fit(x_train_scaled, y_train, epochs=10)

Epoch 1/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 56s 3ms/step - accuracy: 0.6204 - loss: 0.6605
Epoch 2/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 65s 2ms/step - accuracy: 0.6255 - loss: 0.6563
Epoch 3/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - accuracy: 0.6277 - loss: 0.6552
Epoch 4/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.6269 - loss: 0.6554
Epoch 5/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - accuracy: 0.6268 - loss: 0.6554
Epoch 6/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - accuracy: 0.6262 - loss: 0.6556
Epoch 7/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - accuracy: 0.6262 - loss: 0.6560
Epoch 8/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.6279 - loss: 0.6550
Epoch 9/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.6278 - loss: 0.6551
Epoch 10/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - accuracy: 0.6281 - loss: 0.6542


In [69]:
model_1.save("credit_risk_model_1.keras")

In [71]:
model_loss, model_accuracy = model_1.evaluate(x_test_sclaed,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

6076/6076 - 10s - 2ms/step - accuracy: 0.6275 - loss: 0.6548
Loss: 0.6547749638557434, Accuracy: 0.627498984336853


### Summary

This model recorded an accuracy of 62% and a loss of 65%. This leaves room for improvement.

## Second Model Configuration with Additional Layers

In [72]:
#Optimize Model
model_2 = tf.keras.models.Sequential()
model_2.add(tf.keras.layers.Dense(units=1, activation="relu", input_dim=len(df.columns)-1))
model_2.add(tf.keras.layers.Dense(units=20, activation="tanh"))
model_2.add(tf.keras.layers.Dense(units=20, activation="relu"))
model_2.add(tf.keras.layers.Dense(units=20, activation="tanh"))
model_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))



# Compile the Sequential model together and customize metrics
model_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [73]:
model_2_fitted = model_2.fit(x_train_scaled, y_train, epochs=10)

Epoch 1/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 50s 3ms/step - accuracy: 0.6116 - loss: 0.6637
Epoch 2/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 80s 2ms/step - accuracy: 0.6250 - loss: 0.6563
Epoch 3/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 45s 2ms/step - accuracy: 0.6238 - loss: 0.6566
Epoch 4/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 46s 3ms/step - accuracy: 0.6253 - loss: 0.6559
Epoch 5/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 47s 3ms/step - accuracy: 0.6254 - loss: 0.6564
Epoch 6/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 46s 3ms/step - accuracy: 0.6240 - loss: 0.6568
Epoch 7/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 46s 2ms/step - accuracy: 0.6257 - loss: 0.6559
Epoch 8/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 46s 3ms/step - accuracy: 0.6261 - loss: 0.6557
Epoch 9/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 47s 3ms/step - accuracy: 0.6255 - loss: 0.6560
Epoch 10/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - accuracy: 0.6257 - loss: 0.6559


In [74]:
model_2.save("credit_risk_model_2.keras")

In [75]:
model_loss, model_accuracy = model_2.evaluate(x_test_sclaed,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

6076/6076 - 12s - 2ms/step - accuracy: 0.6252 - loss: 0.6557
Loss: 0.655724048614502, Accuracy: 0.6252359747886658


We don't notice much of an increase in our accuracy between the first and second model. We may need to attempt larger optimizations.

## Third Model Configuration with Larger Layers and different activation functions

In [76]:
model_3 = tf.keras.models.Sequential()
model_3.add(tf.keras.layers.Dense(units=32, activation="relu", input_dim=len(df.columns)-1))
model_3.add(tf.keras.layers.Dense(units=100, activation="tanh"))
model_3.add(tf.keras.layers.Dense(units=150, activation="relu"))
model_3.add(tf.keras.layers.Dense(units=100, activation="tanh"))
model_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))



# Compile the Sequential model together and customize metrics
model_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [77]:
model_3_fitted = model_3.fit(x_train_scaled, y_train, epochs=10)

Epoch 1/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 66s 3ms/step - accuracy: 0.6262 - loss: 0.6551
Epoch 2/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 58s 3ms/step - accuracy: 0.6444 - loss: 0.6372
Epoch 3/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - accuracy: 0.6554 - loss: 0.6236
Epoch 4/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 84s 3ms/step - accuracy: 0.6667 - loss: 0.6123
Epoch 5/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 80s 3ms/step - accuracy: 0.6781 - loss: 0.5999
Epoch 6/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 60s 3ms/step - accuracy: 0.6843 - loss: 0.5912
Epoch 7/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 83s 3ms/step - accuracy: 0.6908 - loss: 0.5841
Epoch 8/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 70s 4ms/step - accuracy: 0.6949 - loss: 0.5793
Epoch 9/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 72s 3ms/step - accuracy: 0.6986 - loss: 0.5743
Epoch 10/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 84s 3ms/step - accuracy: 0.7022 - loss: 0.5696


In [79]:
model_3.save("credit_risk_model_3.keras")

### Summary

We noticed a significant increase in our accuracy. It improved by 8% and our Loss decreased by 9%.

If we add some additional layers, we may be able to hit out 75% threshold.

## Model 4 Configuration with even more neurons and an additional layer. Testing with kernal regularization.

In [54]:
model_4 = tf.keras.models.Sequential()
model_4.add(tf.keras.layers.Dense(units=len(df.columns)-1, activation="relu", input_dim=len(df.columns)-1))
model_4.add(tf.keras.layers.Dense(units=100, activation="tanh"))
model_4.add(tf.keras.layers.Dense(units=150, activation="relu"))
model_4.add(tf.keras.layers.Dense(units=150, activation="tanh"))
model_4.add(tf.keras.layers.Dense(units=150, activation="relu"))
model_4.add(tf.keras.layers.Dense(units=64, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model_4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
model_4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [55]:
model_4_fitted = model_4.fit(x_train_scaled, y_train, epochs=10)

Epoch 1/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 62s 3ms/step - accuracy: 0.6248 - loss: 0.6754
Epoch 2/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 60s 3ms/step - accuracy: 0.6448 - loss: 0.6372
Epoch 3/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 83s 3ms/step - accuracy: 0.6641 - loss: 0.6141
Epoch 4/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 62s 3ms/step - accuracy: 0.6772 - loss: 0.5961
Epoch 5/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 83s 3ms/step - accuracy: 0.6884 - loss: 0.5820
Epoch 6/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 83s 4ms/step - accuracy: 0.6941 - loss: 0.5734
Epoch 7/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 62s 3ms/step - accuracy: 0.7009 - loss: 0.5662
Epoch 8/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 79s 3ms/step - accuracy: 0.7035 - loss: 0.5607
Epoch 9/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 60s 3ms/step - accuracy: 0.7082 - loss: 0.5564
Epoch 10/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - accuracy: 0.7103 - loss: 0.5525


In [63]:
model_4.save("credit_risk_model_final.keras")

### Summary

We achieved a slightly higher score after some more tuning. Recording a 71% accuracy and 55% loss. 

Due to time constraints, let's attempt one more to see if we can get a higher score.

In [59]:
model_5 = tf.keras.models.Sequential()
model_5.add(tf.keras.layers.Dense(units=len(df.columns)-1, activation="relu", input_dim=len(df.columns)-1))
model_5.add(tf.keras.layers.Dense(units=100, activation="tanh"))
model_5.add(tf.keras.layers.Dense(units=150, activation="relu"))
model_5.add(tf.keras.layers.Dense(units=150, activation="tanh"))
model_5.add(tf.keras.layers.Dropout(0.2))  # Add dropout with a rate of 0.2
model_5.add(tf.keras.layers.Dense(units=150, activation="relu"))
model_5.add(tf.keras.layers.Dense(units=64, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model_5.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
model_5.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [60]:
model_5_fitted =  model_5.fit(x_train_scaled, y_train, epochs=10)

Epoch 1/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 66s 3ms/step - accuracy: 0.6281 - loss: 0.6758
Epoch 2/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - accuracy: 0.6494 - loss: 0.6342
Epoch 3/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 85s 4ms/step - accuracy: 0.6649 - loss: 0.6143
Epoch 4/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 62s 3ms/step - accuracy: 0.6787 - loss: 0.5982
Epoch 5/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 63s 3ms/step - accuracy: 0.6893 - loss: 0.5850
Epoch 6/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 81s 3ms/step - accuracy: 0.6961 - loss: 0.5762
Epoch 7/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 63s 3ms/step - accuracy: 0.7016 - loss: 0.5697
Epoch 8/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 81s 3ms/step - accuracy: 0.7053 - loss: 0.5647
Epoch 9/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 63s 3ms/step - accuracy: 0.7074 - loss: 0.5605
Epoch 10/10
18228/18228 ━━━━━━━━━━━━━━━━━━━━ 84s 4ms/step - accuracy: 0.7096 - loss: 0.5574


In [62]:
model_5.save("credit_risk_model_5.keras")

### Summary

Our final model did not produce any higher score. Our best model is model 4.

# All Models

Our final model’s overall accuracy is 71%, with a loss of 55%. 

71% accuracy means the model correctly identifies whether a loan applicant is high-risk or low-risk 71% of the time.

A 55% loss suggests that there is still a considerable error in the model's predictions. For risk assessment, this means the model is making mistakes in assessing the risk level of applicants, which could lead to incorrect identification.

